# **Handwriting Text Detection**

In [0]:
import warnings
warnings.filterwarnings("ignore")

The program contains few lines compatible with Tensorflow v2 and few lines with v1. I found that version 1.14.0 is compatible with both sets of lines. Thus uninstalling updated Tensorflow, and installing v1.14.0

In [0]:
!pip uninstall tensorflow
!pip install tensorflow==1.14.0

In [26]:
from __future__ import division
from __future__ import print_function

import sys
import argparse
import cv2
import editdistance # used to calculate errors in characters between predicted and actual words
sys.path.append('/content/drive/My Drive/src')
from DataLoader import DataLoader, Batch # Used for loading data which corresponds to IAM format
from Model import Model, DecoderType # TensorFlow Model for Handwriting Detection
from SamplePreprocessor import preprocess # Preprocessing/cleaning images such as stretching,normalizing the images
import tensorflow as tf
tf.__version__
#/content/drive/My Drive/src/DataLoader.py

'1.14.0'

In [0]:
class FilePaths:
	"filenames and paths to data"
	fnCharList = '/content/drive/My Drive/model/charList.txt'
	fnAccuracy = '/content/drive/My Drive/model/accuracy.txt'
	fnTrain = '/content/drive/My Drive/data/'
	fnInfer = '/content/drive/My Drive/data/test.png'
	fnCorpus = '/content/drive/My Drive/data/corpus.txt'

In [0]:
def train(model, loader):
	"train NN"
	epoch = 0 # number of training epochs since start
	bestCharErrorRate = float('inf') # best valdiation character error rate
	noImprovementSince = 0 # number of epochs no improvement of character error rate occured
	earlyStopping = 2 # stop training after this number of epochs without improvement
	while True:
		epoch += 1
		print('Epoch:', epoch)

		# train
		print('Train NN')
		loader.trainSet()
		while loader.hasNext():
			iterInfo = loader.getIteratorInfo()
			batch = loader.getNext()
			loss = model.trainBatch(batch)
			print('Batch:', iterInfo[0],'/', iterInfo[1], 'Loss:', loss)

		# validate
		charErrorRate = validate(model, loader)
		
		# if best validation accuracy so far, save model parameters
		if charErrorRate < bestCharErrorRate:
			print('Character error rate improved, save model')
			bestCharErrorRate = charErrorRate
			noImprovementSince = 0
			model.save()
			open(FilePaths.fnAccuracy, 'w').write('Validation character error rate of saved model: %f%%' % (charErrorRate*100.0))
		else:
			print('Character error rate not improved')
			noImprovementSince += 1

		# stop training if no more improvement in the last x epochs
		if noImprovementSince >= earlyStopping:
			print('No more improvement since %d epochs. Training stopped.' % earlyStopping)
			break

In [0]:
def validate(model, loader):
	"validate NN"
	print('Validate NN')
	loader.validationSet()
	numCharErr = 0
	numCharTotal = 0
	numWordOK = 0
	numWordTotal = 0
	while loader.hasNext():
		iterInfo = loader.getIteratorInfo()
		print('Batch:', iterInfo[0],'/', iterInfo[1])
		batch = loader.getNext()
		(recognized, _) = model.inferBatch(batch)
		
		print('Ground truth -> Recognized')	
		for i in range(len(recognized)):
			numWordOK += 1 if batch.gtTexts[i] == recognized[i] else 0
			numWordTotal += 1
			dist = editdistance.eval(recognized[i], batch.gtTexts[i])
			numCharErr += dist
			numCharTotal += len(batch.gtTexts[i])
			print('[OK]' if dist==0 else '[ERR:%d]' % dist,'"' + batch.gtTexts[i] + '"', '->', '"' + recognized[i] + '"')
	
	# print validation result
	charErrorRate = numCharErr / numCharTotal
	wordAccuracy = numWordOK / numWordTotal
	print('Character error rate: %f%%. Word accuracy: %f%%.' % (charErrorRate*100.0, wordAccuracy*100.0))
	return charErrorRate

In [0]:
def infer(model, fnImg):
	"recognize text in image provided by file path"
	img = preprocess(cv2.imread(fnImg, cv2.IMREAD_GRAYSCALE), Model.imgSize)
	batch = Batch(None, [img])
	(recognized, probability) = model.inferBatch(batch, True)
	print('Recognized:', '"' + recognized[0] + '"')
	print('Probability:', probability[0])

In [0]:
def train_model():
  print(FilePaths.fnTrain)
  loader = DataLoader(FilePaths.fnTrain, Model.batchSize, Model.imgSize, Model.maxTextLen)
  open(FilePaths.fnCharList, 'w').write(str().join(loader.charList))
  open(FilePaths.fnCorpus, 'w').write(str(' ').join(loader.trainWords + loader.validationWords))
  model = Model(loader.charList, decoderType)
  train(model, loader)

In [0]:
train_model()

In [0]:
def validate_model():
  loader = DataLoader(FilePaths.fnTrain, Model.batchSize, Model.imgSize, Model.maxTextLen)
  open(FilePaths.fnCharList, 'w').write(str().join(loader.charList))
  open(FilePaths.fnCorpus, 'w').write(str(' ').join(loader.trainWords + loader.validationWords))
  model = Model(loader.charList, decoderType, mustRestore=True)
  validate(model, loader)

In [0]:
def infer_model(decoderType,dump_val): #Infering/testing the model
  print(open(FilePaths.fnAccuracy).read())
  model = Model(open(FilePaths.fnCharList).read(), decoderType, mustRestore=True, dump=dump_val)
  infer(model, FilePaths.fnInfer)

In [0]:
print("Enter which decoding algorithm to use:")
decoderType = int(input("BestPath - 0, BeamSearch - 1( Enter 0 or 1):    "))
print(type(decoderType))

Enter which decoding algorithm to use:
BestPath - 0, BeamSearch - 1, WordBeamSearch - 2 ( Enter 0 or 1 or 2):    0
<class 'int'>


In [32]:
print("Do you want to dump output of NN to CSV file(s)")
dump_str = input("Yes or No:  ")
if dump_str=="Yes" or dump_str=="yes" or dump_str=="YES":
  dump_value=True
elif dump_str=="No" or dump_str=="no" or dump_str=="NO":
  dump_value=False

infer_model(decoderType,dump_value)

Do you want to dump output of NN to CSV file(s)
Yes or No:  no
Validation character error rate of saved model: 15.750033%
Python: 3.6.9 (default, Nov  7 2019, 10:44:02) 
[GCC 8.3.0]
Tensorflow: 1.14.0
Init with stored values from /content/drive/My Drive/model/snapshot-2
INFO:tensorflow:Restoring parameters from /content/drive/My Drive/model/snapshot-2
Recognized: "little"
Probability: 0.52248619


In [0]:
tf.__version__